In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [10]:
if torch.cuda.is_available():
    device = 'cuda'

In [3]:
for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_name(i))

NVIDIA GeForce RTX 4070


In [4]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¹', '‒', '—', '―', '‘', '’', '“', '”', '•', '™', '♠', '♦', '\ufeff']
96


In [7]:
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for idx, char in enumerate(chars)}

In [12]:
print(idx_to_char)

{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '$', 5: '%', 6: '&', 7: "'", 8: '(', 9: ')', 10: '*', 11: ',', 12: '-', 13: '.', 14: '/', 15: '0', 16: '1', 17: '2', 18: '3', 19: '4', 20: '5', 21: '6', 22: '7', 23: '8', 24: '9', 25: ':', 26: ';', 27: '?', 28: 'A', 29: 'B', 30: 'C', 31: 'D', 32: 'E', 33: 'F', 34: 'G', 35: 'H', 36: 'I', 37: 'J', 38: 'K', 39: 'L', 40: 'M', 41: 'N', 42: 'O', 43: 'P', 44: 'Q', 45: 'R', 46: 'S', 47: 'T', 48: 'U', 49: 'V', 50: 'W', 51: 'X', 52: 'Y', 53: 'Z', 54: '[', 55: ']', 56: '_', 57: 'a', 58: 'b', 59: 'c', 60: 'd', 61: 'e', 62: 'f', 63: 'g', 64: 'h', 65: 'i', 66: 'j', 67: 'k', 68: 'l', 69: 'm', 70: 'n', 71: 'o', 72: 'p', 73: 'q', 74: 'r', 75: 's', 76: 't', 77: 'u', 78: 'v', 79: 'w', 80: 'x', 81: 'y', 82: 'z', 83: '¹', 84: '‒', 85: '—', 86: '―', 87: '‘', 88: '’', 89: '“', 90: '”', 91: '•', 92: '™', 93: '♠', 94: '♦', 95: '\ufeff'}


In [14]:
data = [char_to_idx[char] for char in text ]

In [17]:
if len(data) == len(text):
    print(f"True! Length is {len(data)}")

True! Length is 725438


In [20]:
inputs = []
targets = []
seq_length = 100

for i in range(0, len(data) -seq_length):
    inputs.append(data[i:i+seq_length])
    targets.append(data[i+1:i+seq_length+1])

inputs = torch.tensor(inputs, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

In [28]:
print(inputs.size())
print(targets.size())
inputs.to(device)
targets.to(device)

torch.Size([725338, 100])
torch.Size([725338, 100])


tensor([[31, 42, 45,  ..., 11,  1, 42],
        [42, 45, 42,  ...,  1, 42, 53],
        [45, 42, 47,  ..., 42, 53, 40],
        ...,
        [71,  1, 64,  ..., 71, 67, 75],
        [ 1, 64, 61,  ..., 67, 75, 13],
        [64, 61, 68,  ..., 75, 13,  0]], device='cuda:0')

In [33]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        # self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.rnn(x, hidden)
        # out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)

# Model parameters
input_size = vocab_size
hidden_size = 16
output_size = vocab_size

# Initialize model, loss function, and optimizer
model = SimpleRNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [36]:
init_hidden = model.init_hidden(inputs.size(0)) 
out, hidden = model(inputs, init_hidden)

In [37]:
out.size()

torch.Size([725338, 100, 16])